In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import display, Audio

# Helper function to play a beep sound
def play_simple_beep():
    framerate = 44100
    duration = 1
    t = np.linspace(0, duration, int(framerate * duration))
    sound = np.sin(2 * np.pi * 440 * t)
    display(Audio(sound, rate=framerate, autoplay=True))

# Define the neural network with flexible hidden layers
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_layers=[128, 64, 32]):
        super(NeuralNetwork, self).__init__()
        layers = []
        for i, hidden_size in enumerate(hidden_layers):
            layers.append(nn.Linear(input_size if i == 0 else hidden_layers[i-1], hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.LeakyReLU(negative_slope=0.01))
        layers.append(nn.Linear(hidden_layers[-1], 1))  # Output layer
        layers.append(nn.Sigmoid())  # Sigmoid activation for binary classification
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)


# # Data preparation
# scaler = StandardScaler()
# X_train_np = scaler.fit_transform(X_train)  # Using NumPy arrays directly
# X_test_np = scaler.transform(X_test)       # Using NumPy arrays directly

# # Convert to PyTorch Tensor
# X_train = torch.tensor(X_train_np, dtype=torch.float32)
# X_test = torch.tensor(X_test_np, dtype=torch.float32)
# y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32).view(-1, 1)
# y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32).view(-1, 1)


# Hyperparameters
config = {
    "input_size": X_train.shape[1],
    "hidden_layers": [128, 64,32],
    "batch_size": 32,
    "lr": 0.001,
    "weight_decay": 0.001,
    "epochs": 500,
    "patience": 20
}

# Create DataLoader for training
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize the model, loss function, optimizer, and scheduler
model = NeuralNetwork(input_size=config["input_size"], hidden_layers=config["hidden_layers"]).to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Training with early stopping
start_time = time.time()
best_val_loss = np.inf
early_stopping_counter = 0
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(config["epochs"]):
    model.train()
    epoch_loss = 0
    correct_train = 0
    total_train = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
        # Calculate training accuracy
        predicted_train = (y_pred > 0.5).float()
        correct_train += (predicted_train == y_batch).sum().item()
        total_train += y_batch.size(0)
    
    train_losses.append(epoch_loss / len(train_loader))
    train_accuracies.append(correct_train / total_train)
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_pred = model(X_test.to(device))
        val_loss = criterion(val_pred, y_test.to(device))
        val_losses.append(val_loss.item())
        predicted_val = (val_pred > 0.5).float()
        val_accuracy = accuracy_score(y_test.cpu(), predicted_val.cpu())
        val_accuracies.append(val_accuracy)

        # Reduce LR and track best model
        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
            best_model_state = model.state_dict()
        else:
            early_stopping_counter += 1
        
        # Check for early stopping
        if early_stopping_counter >= config["patience"]:
            print("Early stopping triggered.")
            break

# Load the best model
model.load_state_dict(best_model_state)

# Plot training and validation metrics
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label="Training Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate on test set
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test.to(device))
    y_test_pred_class = (y_test_pred > 0.5).float()
    test_accuracy = accuracy_score(y_test.cpu(), y_test_pred_class.cpu())
    print(f"Test Accuracy: {test_accuracy:.5f}")

    # Display confusion matrix
    cm = confusion_matrix(y_test.cpu(), y_test_pred_class.cpu())
    ConfusionMatrixDisplay(cm, display_labels=["Class 0", "Class 1"]).plot()
    plt.title("Confusion Matrix")
    plt.show()

# Play beep sound and print execution time
play_simple_beep()
print(f"Total execution time: {time.time() - start_time:.2f} seconds")
